
# ハードウェア

:label: `sec_hardware`

優れたパフォーマンスのシステムを構築するには、問題の統計的側面を捉えるためのアルゴリズムとモデルをよく理解する必要があります。同時に、基盤となるハードウェアについて少なくともある程度の知識を持っていることも不可欠です。このセクションは、ハードウェアおよびシステム設計に関する適切なコースの代わりにはなりません。むしろ、一部のアルゴリズムが他のアルゴリズムより効率的である理由や、良好なスループットを達成する方法を理解するための出発点として機能する可能性があります。優れた設計は簡単に桁違いの違いを生み出すことができ、その結果、ネットワークをトレーニングできる (たとえば 1 週間以内) か、まったくできない (3 か月以内で期限に間に合わない) かの違いが生じる可能性があります。 ）。まずはコンピュータから見ていきます。次に、ズームインして CPU と GPU をさらに注意深く見ていきます。最後に、ズームアウトして、サーバー センターまたはクラウドで複数のコンピューターがどのように接続されているかを確認します。 

![](../img/latencynumbers.png) :label: `fig_latencynumbers`

せっかちな読者は :numref: `fig_latencynumbers`でなんとかなるかもしれません。これは、過去 10 年間の進歩の概要を示す Colin Scott の[インタラクティブな投稿](https://people.eecs.berkeley.edu/%7Ercs/research/interactive_latency.html)から引用したものです。元の数字は、 [2010 年のジェフ ディーンのスタンフォードでの講演](https://static.googleusercontent.com/media/research.google.com/en//people/jeff/Stanford-DL-Nov-2010.pdf)によるものです。以下の説明では、これらの数値の理論的根拠の一部と、それらの数値がアルゴリズムの設計にどのように役立つかを説明します。以下の説明は非常に高レベルで大雑把なものです。これは明らかに適切なコースの*代わりとなるもので*はなく、統計モデラーが適切な設計上の決定を下すのに十分な情報を提供することを目的としています。コンピュータ アーキテクチャの詳細な概要については、:cite: `Hennessy.Patterson.2011` 、または[Arste Asanovic](http://inst.eecs.berkeley.edu/%7Ecs152/sp19/)によるコースなど、このテーマに関する最近のコースを参照してください。

## コンピュータ

ほとんどの深層学習の研究者や実践者は、かなりの量のメモリ、計算機能、GPU などの何らかの形式のアクセラレータ、またはそれらの倍数を備えたコンピュータにアクセスできます。コンピューターは次の主要なコンポーネントで構成されています。
-  (オペレーティング システムやその他多くのものの実行に加えて) 与えられたプログラムを実行できるプロセッサ (CPU とも呼ばれます)。通常は 8 個以上のコアで構成されます。
- 重みベクトルやアクティベーション、トレーニング データなどの計算結果を保存および取得するためのメモリ (RAM)。
-  1 GB/秒から 100 GB/秒の範囲の速度を持つイーサネット ネットワーク接続 (場合によっては複数)。ハイエンド サーバーでは、より高度な相互接続が見つかります。
- システムを 1 つ以上の GPU に接続するための高速拡張バス (PCIe)。サーバーには最大 8 つのアクセラレータがあり、高度なトポロジで接続されることがよくありますが、デスクトップ システムには、ユーザーの予算と電源のサイズに応じて 1 つまたは 2 つのアクセラレータがあります。
- 磁気ハードディスク ドライブやソリッド ステート ドライブなどの耐久性のあるストレージは、多くの場合 PCIe バスを使用して接続されます。これにより、トレーニング データをシステムに効率的に転送し、必要に応じて中間チェックポイントを保存できます。 

![](../img/mobo-symbol.svg):label: `fig_mobo-symbol`

 :numref: `fig_mobo-symbol`が示すように、ほとんどのコンポーネント (ネットワーク、GPU、ストレージ) は PCIe バスを介して CPU に接続されています。 CPU に直接接続された複数のレーンで構成されます。たとえば、AMD の Threadripper 3 には 64 個の PCIe 4.0 レーンがあり、各レーンは両方向に 16 Gbit/s のデータ転送が可能です。メモリは CPU に直接接続されており、総帯域幅は最大 100 GB/秒です。

コンピューター上でコードを実行するときは、データをプロセッサー (CPU または GPU) にシャッフルし、計算を実行し、その結果をプロセッサーから RAM および耐久性のあるストレージに戻す必要があります。したがって、優れたパフォーマンスを得るには、どのシステムも大きなボトルネックになることなく、これがシームレスに機能することを確認する必要があります。たとえば、画像を十分に速くロードできない場合、プロセッサは何も行うことができません。同様に、行列を CPU (または GPU) に十分な速さで移動できない場合、その処理要素が枯渇してしまいます。最後に、ネットワーク上で複数のコンピュータを同期したい場合、後者によって計算が遅くなってはなりません。 1 つのオプションは、通信と計算をインターリーブすることです。さまざまなコンポーネントをさらに詳しく見てみましょう。



## メモリー

最も基本的なメモリは、すぐにアクセスできる必要があるデータを保存するために使用されます。現在、CPU RAM は通常[DDR4](https://en.wikipedia.org/wiki/DDR4_SDRAM)タイプであり、モジュールあたり 20 ～ 25 GB/秒の帯域幅を提供します。各モジュールには 64 ビット幅のバスがあります。通常、複数のチャネルを可能にするためにメモリ モジュールのペアが使用されます。 CPU には 2 ～ 4 個のメモリ チャネルがあります。つまり、4 0GB/s ～ 100 GB/s のピーク メモリ帯域幅があります。多くの場合、チャネルごとに 2 つのバンクがあります。たとえば、AMD の Zen 3 Threadripper には 8 つのスロットがあります。

これらの数字は印象的なものではありますが、実際、これらは物語の一部にすぎません。メモリから一部を読み出したいときは、まず情報がどこにあるかをメモリ モジュールに伝える必要があります。つまり、最初に*アドレスを*RAM に送信する必要があります。これが完了すると、単一の 64 ビット レコードだけを読み取るか、長いレコード シーケンスを読み取るかを選択できます。後者は*バースト読み取り*と呼ばれます。簡単に言うと、メモリへのアドレスの送信と転送のセットアップには約 100 ns かかります (詳細は、使用するメモリ チップの特定のタイミング係数によって異なります)。その後の転送にはわずか 0.2 ns しかかかりません。つまり、最初の読み取りはその後の読み取りの 500 倍の費用がかかります。 1 秒あたり最大 10,000,000 回のランダム読み取りを実行できることに注意してください。これは、ランダム メモリ アクセスを可能な限り回避し、代わりにバースト読み取り (および書き込み) を使用することを示唆しています。

複数の*銀行が*あることを考慮すると、問題はもう少し複雑になります。各バンクはほぼ独立してメモリを読み取ることができます。これは 2 つのことを意味します。一方で、メモリ全体に均等に分散されている場合、ランダム読み取りの有効回数は最大 4 倍になります。また、バースト読み取りも 4 倍高速であるため、ランダム読み取りを実行するのは依然として悪い考えであることも意味します。一方、メモリは 64 ビット境界に合わせて配置されるため、データ構造を同じ境界に合わせることをお勧めします。適切なフラグが設定されている場合、コンパイラーはこれをほぼ[自動的に](https://en.wikipedia.org/wiki/Data_structure_alignment)実行します。興味のある読者は、 [Zeshan Chishti](http://web.cecs.pdx.edu/%7Ezeshan/ece585_lec5.pdf)による DRAM に関する講義などを参照することをお勧めします。

 GPU メモリには CPU よりも多くの処理要素が搭載されているため、さらに高い帯域幅要件が課せられます。これらに対処するには、概して 2 つのオプションがあります。 1 つ目は、メモリ バスを大幅に広くすることです。たとえば、NVIDIA の RTX 2080 Ti には 352 ビット幅のバスがあります。これにより、より多くの情報を同時に転送できるようになります。次に、GPU は特定の高性能メモリを使用します。 NVIDIA の RTX や Titan シリーズなどのコンシューマー グレードのデバイスは通常、総帯域幅が 500 GB/秒を超える[GDDR6](https://en.wikipedia.org/wiki/GDDR6_SDRAM)チップを使用します。代わりに、HBM (高帯域幅メモリ) モジュールを使用することもできます。これらは非常に異なるインターフェイスを使用し、専用のシリコン ウェーハ上の GPU に直接接続します。そのため、それらは非常に高価になり、その使用は通常、NVIDIA Volta V100 シリーズ アクセラレータなどのハイエンド サーバー チップに限定されます。当然のことですが、GPU メモリはコストが高いため、一般に CPU メモリよりもはるかに*小さく*なります。私たちの目的では、それらのパフォーマンス特性は概して同様ですが、はるかに高速です。本書の目的上、詳細は無視しても問題ありません。これらは、高スループットを実現するために GPU カーネルを調整する場合にのみ重要です。

## 保管所

RAM の重要な特性の一部が*帯域幅*と*遅延*であることがわかりました。ストレージ デバイスにも同じことが当てはまりますが、違いはさらに極端になる可能性があります。

### ハード・ディスク・ドライブ

*ハードディスク ドライブ*(HDD) は半世紀以上使用されています。簡単に言うと、任意のトラックで読み取りまたは書き込みを行うために配置できるヘッドを備えた多数の回転プラッターが含まれています。ハイエンド ディスクは、9 プラッターで最大 16 TB を保持します。 HDD の主な利点の 1 つは、比較的安価であることです。多くの欠点のうちの 1 つは、一般的に致命的な障害モードが発生することと、読み取りレイテンシが比較的高いことです。

後者を理解するには、HDD が約 7,200 RPM (1 分あたりの回転数) で回転するという事実を考慮してください。それがはるかに速い場合は、大皿にかかる遠心力によって粉々になってしまうでしょう。これには、ディスク上の特定のセクターにアクセスする場合に大きな欠点があります。プラッターが所定の位置で回転するまで待つ必要があります (ヘッドを移動することはできますが、実際のディスクを加速することはできません)。したがって、要求されたデータが利用可能になるまでに 8 ミリ秒以上かかる場合があります。これを表す一般的な方法は、HDD が約 100 IOP (1 秒あたりの入出力操作) で動作できるということです。この数字は過去 20 年間、基本的に変わっていません。さらに悪いことに、帯域幅を増やすことも同様に困難です (帯域幅は 100 ～ 200 MB/秒程度です)。結局のところ、各ヘッドはビットのトラックを読み取るため、ビット レートは情報密度の平方根にのみ対応します。その結果、HDD は急速にアーカイブ ストレージや、非常に大規模なデータセット用の低グレード ストレージに追いやられつつあります。

### ソリッドステートドライブ

ソリッド ステート ドライブ (SSD) はフラッシュ メモリを使用して情報を永続的に保存します。これにより、保存されたレコードへのアクセスが*大幅に高速*化されます。最新の SSD は 100,000 ～ 500,000 IOP で動作でき、これは HDD よりも最大 3 桁高速です。さらに、その帯域幅は 1 ～ 3GB/s に達する可能性があり、つまり HDD よりも 1 桁高速です。これらの改善は、あまりにもうますぎるように聞こえます。実際、SSD の設計方法により、次のような注意事項があります。
-  SSD は情報をブロック (256 KB 以上) に保存します。全体としてしか書けないので、かなりの時間がかかります。したがって、SSD でのビット単位のランダム書き込みのパフォーマンスは非常に低下します。同様に、データの書き込みには、ブロックを読み取って消去し、新しい情報で再書き込みする必要があるため、一般にかなりの時間がかかります。現在、SSD コントローラーとファームウェアは、これを軽減するアルゴリズムを開発しています。それにもかかわらず、特に QLC (クアッド レベル セル) SSD の場合、書き込みが大幅に遅くなることがあります。パフォーマンスを向上させるための鍵は、操作の*キューを*維持し、読み取りを優先し、可能であれば大きなブロックに書き込むことです。
-  SSD のメモリ セルは比較的早く消耗します (多くの場合、数千回の書き込み後にすでに消耗します)。摩耗レベルの保護アルゴリズムにより、劣化が多くのセルに広がる可能性があります。ただし、ファイルの交換やログ ファイルの大規模な集合に SSD を使用することはお勧めできません。
- 最後に、帯域幅の大幅な増加により、コンピューター設計者は SSD を PCIe バスに直接接続する必要がありました。これを処理できるドライブは NVMe (Non Volatile Memory Enhanced) と呼ばれ、最大 4 つの PCIe レーンを使用できます。これは、PCIe 4.0 では最大 8GB/s に相当します。

### クラウドストレージ

クラウド ストレージは、構成可能なパフォーマンスの範囲を提供します。つまり、仮想マシンへのストレージの割り当ては、量と速度の両方の点で、ユーザーの選択に従って動的に行われます。多数の小さなレコードを使用したトレーニング中など、レイテンシが高すぎる場合には、プロビジョニングされた IOP の数を増やすことをお勧めします。



## CPU

中央処理装置 (CPU) は、あらゆるコンピューターの中心部分です。これらは、マシン コードを実行できる*プロセッサ コア*、プロセッサ コアを接続する*バス*(特定のトポロジはプロセッサ モデル、世代、ベンダーによって大きく異なります)、およびより高い帯域幅とより低いレイテンシのメモリを可能にする*キャッシュなど*、多数の主要なコンポーネントで構成されています。メインメモリからの読み取りよりもアクセスが困難です。最後に、ほとんどすべての最新の CPU には、メディア処理や機械学習で一般的である高性能線形代数や畳み込みを支援する*ベクトル処理ユニットが*含まれています。 

![](http://d2l.ai/_images/skylake.svg) :ラベル: `fig_skylake`

 :numref: `fig_skylake` Intel Skylake コンシューマ グレードのクアッドコア CPU を示しています。統合された GPU、キャッシュ、および 4 つのコアを接続するリングバスを備えています。イーサネット、WiFi、Bluetooth、SSD コントローラー、USB などの周辺機器は、チップセットの一部であるか、CPU に直接接続 (PCIe) されています。

### マイクロアーキテクチャ

各プロセッサ コアは、かなり洗練されたコンポーネントのセットで構成されています。詳細は世代やベンダーによって異なりますが、基本的な機能はほぼ標準です。フロントエンドは命令をロードし、どのパスが選択されるかを予測しようとします (制御フローなど)。その後、命令はアセンブリ コードからマイクロ命令にデコードされます。アセンブリ コードは、プロセッサが実行する最低レベルのコードではないことがよくあります。代わりに、複雑な命令は、より低レベルの操作のセットにデコードされる場合があります。これらは実際の実行コアによって処理されます。多くの場合、後者は多くの操作を同時に実行できます。たとえば、:numref: `fig_cortexa77`の ARM Cortex A77 コアは、最大 8 つの操作を同時に実行できます。 

![](http://d2l.ai/_images/a77.svg) :ラベル: `fig_cortexa77`

これは、効率的なプログラムは、独立して実行できる場合には、クロック サイクルごとに複数の命令を実行できる可能性があることを意味します。すべてのユニットが同じように作られているわけではありません。整数命令に特化したものもありますが、浮動小数点のパフォーマンスに最適化されたものもあります。スループットを向上させるために、プロセッサは分岐命令で複数のコード パスを同時にたどり、分岐しなかった結果を破棄することもあります。これが、最も有望なパスのみが追跡されるように、分岐予測ユニットが (フロントエンドで) 重要になる理由です。

### ベクトル化

ディープラーニングは非常に多くのコンピューティングを必要とします。したがって、CPU を機械学習に適したものにするには、1 クロック サイクルで多くの演算を実行する必要があります。これはベクトルユニットによって実現されます。これらにはさまざまな名前があり、ARM では NEON と呼ばれ、x86 (最近の世代) では[AVX2](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions)ユニットと呼ばれます。共通する点は、SIMD (単一命令複数データ) 操作を実行できることです。 :numref: `fig_neon128` ARM で 1 クロック サイクルで 8 つの短整数を加算する方法を示しています。

![](http://d2l.ai/_images/neon128.svg) :ラベル: `fig_neon128`

アーキテクチャの選択に応じて、このようなレジスタは最大 512 ビット長になり、最大 64 組の数値の組み合わせが可能になります。たとえば、2 つの数値を乗算して 3 番目の数値に加算する場合があります。これは融合積和とも呼ばれます。 Intel の[OpenVino は](https://01.org/openvinotoolkit)これらを使用して、サーバー グレードの CPU でディープ ラーニングのかなりのスループットを実現します。ただし、この数字は GPU が達成できる能力に比べればまったく小さく見えないことに注意してください。たとえば、NVIDIA の RTX 2080 Ti には 4,352 個の CUDA コアがあり、それぞれがそのような操作をいつでも処理できます。

### キャッシュ

次の状況を考えてみましょう。上記の :numref: `fig_skylake`に示されているように、4 つのコアを備えた控えめな CPU コアがあり、2 GHz の周波数で実行されています。さらに、IPC (クロックあたりの命令数) カウントが 1 で、ユニットで 256 ビット幅が有効な AVX2 があると仮定します。さらに、AVX2 操作に使用されるレジスタの少なくとも 1 つをメモリから取得する必要があると仮定します。これは、CPU がクロック サイクルごとに $4 \times 256 \text{ bit} = 128 \text{ bytes}$ のデータを消費することを意味します。 1 秒あたり $2 \times 10^9 \times 128 = 256 \times 10^9$ バイトをプロセッサに転送できない限り、処理要素は枯渇してしまいます。残念ながら、このようなチップのメモリ インターフェイスは 20 ～ 40 GB/s、つまり 1 桁少ないデータ転送しかサポートしていません。この修正は、メモリから*新しい*データをロードすることを可能な限り回避し、CPU 上でローカルにキャッシュすることです。ここでキャッシュが役に立ちます。一般に、次の名前または概念が使用されます。
- 厳密に言えば、**レジ​​スタは**キャッシュの一部ではありません。彼らはステージの指示に役立ちます。つまり、CPU レジスタは、CPU が遅延ペナルティなしでクロック速度でアクセスできるメモリの場所です。 CPUには数十のレジスタがあります。レジスタを効率的に使用できるかどうかはコンパイラ (またはプログラマ) の責任です。たとえば、C プログラミング言語には`register`キーワードがあります。
-  **L1 キャッシュは、**高いメモリ帯域幅要件に対する防御の第一線です。 L1 キャッシュは小さく (通常のサイズは 32 ～ 64 KB)、多くの場合、データ キャッシュと命令キャッシュに分割されます。 L1 キャッシュでデータが見つかった場合、アクセスは非常に高速になります。そこで見つからない場合、検索はキャッシュ階層を下って進みます。
-  **L2 キャッシュが**次の目的地です。アーキテクチャ設計とプロセッサ サイズによっては、これらが排他的になる場合があります。これらは、特定のコアによってのみアクセス可能である場合もあれば、複数のコア間で共有される場合もあります。 L2 キャッシュは L1 よりも大きく (通常、コアあたり 256 ～ 512 KB)、低速です。さらに、L2 にあるものにアクセスするには、まずデータが L1 にないことを確認する必要があり、これによりわずかな追加の待ち時間が発生します。
-  **L3 キャッシュは**複数のコア間で共有され、非常に大きくなる可能性があります。 AMD の Epyc 3 サーバー CPU には、複数のチップレットにまたがるなんと 256 MB のキャッシュがあります。より一般的な数値は 4 ～ 8 MB の範囲です。

次にどのメモリ要素が必要になるかを予測することは、チップ設計における重要な最適化パラメータの 1 つです。たとえば、ほとんどのキャッシュ アルゴリズムは*逆方向*ではなく前方*を読み取ろ*うとするため、メモリを順方向にトラバースすることをお勧めします。同様に、メモリ アクセス パターンをローカルに保つことは、パフォーマンスを向上させる良い方法です。

キャッシュの追加は諸刃の剣です。一方で、プロセッサ コアのデータが枯渇しないようにします。同時に、チップ サイズが増加し、処理能力の向上に費やされるはずだった領域が使用されてしまいます。さらに、*キャッシュミスは*大きな損失をもたらす可能性があります。 :numref: `fig_falsesharing`に示されているように、最悪のシナリオである*false Sharing*を考えてみましょう。プロセッサ 1 上のスレッドがデータを要求すると、メモリ位置はプロセッサ 0 にキャッシュされます。それを取得するには、プロセッサ 0 が実行中の処理を停止し、その情報をメイン メモリに書き戻してから、プロセッサ 1 にメモリから情報を読み取らせる必要があります。この操作中、両方のプロセッサは待機します。このようなコードは、効率的な単一プロセッサの実装と比較した場合、複数のプロセッサ上では実行速度*が遅くなる*可能性があります。これは、キャッシュ サイズ (物理サイズ以外に) に実質的な制限があるもう 1 つの理由です。 

![](http://d2l.ai/_images/falsesharing.svg) :label: `fig_falsesharing`

##  GPU およびその他のアクセラレータ

GPU がなければディープラーニングは成功しなかったと言っても過言ではありません。同様に、ディープラーニングのおかげで GPU メーカーの財産が大幅に増加したと主張するのも当然です。このハードウェアとアルゴリズムの共進化により、良くも悪くも深層学習が好ましい統計モデリング パラダイムとなる状況が生まれました。したがって、GPU や TPU などの関連アクセラレータの具体的な利点を理解することは有益です (cite: `Jouppi.Young.Patil.ea.2017` )。

注目すべきは、実際によく行われる区別です。アクセラレータはトレーニングまたは推論のいずれかに最適化されています。後者の場合、ネットワーク内の順方向伝播を計算するだけで済みます。バックプロパゲーションには中間データのストレージは必要ありません。さらに、あまり正確な計算は必要ないかもしれません (通常は FP16 または INT8 で十分です)。一方、トレーニング中は、すべての中間結果に勾配を計算するためのストレージが必要です。さらに、勾配を累積するには、数値のアンダーフロー (またはオーバーフロー) を回避するために、より高い精度が必要です。これは、FP16 (または FP32 との混合精度) が最小要件であることを意味します。これらすべてにより、より高速で大容量のメモリ (HBM2 対 GDDR6) とより多くの処理能力が必要になります。たとえば、NVIDIA の[Turing](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/) T4 GPU は推論用に最適化されていますが、V100 GPU はトレーニング用に適しています。

 :numref: `fig_neon128`に示されているベクトル化を思い出してください。プロセッサ コアにベクトル ユニットを追加することで、スループットを大幅に向上させることができました。たとえば、:numref: `fig_neon128`の例では、16 個の操作を同時に実行できました。まず、ベクトル間の演算だけでなく行列間の演算も最適化する演算を追加したらどうなるでしょうか?この戦略は、テンソル コア (すぐに説明します) につながりました。次に、さらに多くのコアを追加したらどうなるでしょうか?一言で言えば、これら 2 つの戦略は GPU における設計上の決定を要約したものです。 :numref: `fig_turing_processing_block` 、基本的な処理ブロックの概要を示します。これには、16 個の整数ユニットと 16 個の浮動小数点ユニットが含まれます。それに加えて、2 つの tensor コアは、深層学習に関連する追加操作の狭いサブセットを高速化します。各ストリーミング マルチプロセッサは、このような 4 つのブロックで構成されます。 

![](../img/turing-processing-block.png) :幅: `150px` :ラベル: `fig_turing_processing_block`

次に、12 個のストリーミング マルチプロセッサがグラフィックス処理クラスタにグループ化され、ハイエンド TU102 プロセッサが構成されます。十分なメモリ チャネルと L2 キャッシュがセットアップを補完します。 :numref: `fig_turing`に関連する詳細が記載されています。このようなデバイスを設計する理由の 1 つは、必要に応じて個々のブロックを追加または削除して、よりコンパクトなチップを実現し、歩留まりの問題 (障害のあるモジュールがアクティブにならない可能性がある) に対処できることです。幸いなことに、このようなデバイスのプログラミングは、CUDA とフレームワーク コードの層の下にある一般の深層学習研究者には十分に隠されています。特に、利用可能なリソースがあれば、複数のプログラムが GPU 上で同時に実行される可能性があります。それにもかかわらず、デバイスのメモリに適合しないモデルを選択することを避けるために、デバイスの制限を認識することは有益です。 

![](../img/turing.png) :幅: `350px` :ラベル: `fig_turing`

さらに詳しく言及する価値のある最後の側面は、*テンソル コア*です。これらは、ディープ ラーニングに特に効果的な、より最適化された回路を追加するという最近の傾向の一例です。たとえば、TPU は高速行列乗算のためにシストリック配列 :cite: `Kung.1988`を追加しました。そこでの設計は、非常に少数 (第 1 世代の TPU に 1 つ) の大規模な操作をサポートすることでした。 Tensor コアはもう一方の端にあります。これらは、数値精度に応じて $4 \times 4$ から $16 \times 16$ までの行列を含む小規模な演算用に最適化されています。 :numref: `fig_tensorcore`最適化の概要を示します。 

![](../img/tensorcore.jpg) :幅: `400px` :ラベル: `fig_tensorcore`

明らかに、計算を最適化する際には、ある程度の妥協をすることになります。その 1 つは、GPU が割り込みやスパース データの処理があまり得意ではないということです。 [Gunrock](https://github.com/gunrock/gunrock) :cite: `Wang.Davidson.Pan.ea.2016`などの注目すべき例外はありますが、スパース行列とベクトルのアクセス パターンは、GPU が優れている高帯域幅のバースト読み取り操作とはうまくいきません。両方の目標を一致させることは、活発な研究分野です。たとえば、グラフの深層学習用に調整されたライブラリである[DGL](http://dgl.ai)を参照してください。



## ネットワークとバス

単一のデバイスでは最適化が不十分な場合は常に、処理を同期するためにそのデバイスとの間でデータを転送する必要があります。ここでネットワークとバスが役に立ちます。帯域幅、コスト、距離、柔軟性など、多くの設計パラメータがあります。一方では、かなり範囲が広く、非常に使いやすく (結局のところ、配線がありません)、安い WiFi がありますが、帯域幅と遅延は比較的平凡です。まともな考えを持っている機械学習研究者であれば、サーバーのクラスターを構築するために機械学習を使用することはありません。以下では、深層学習に適した相互接続に焦点を当てます。
-  **PCIe は、**レーンごとの非常に高帯域幅のポイントツーポイント接続 (16 レーン スロットの PCIe 4.0 で最大 32 GB/秒) のための専用バスです。遅延は 1 桁のマイクロ秒 (5 μs) のオーダーです。 PCIe リンクは貴重です。プロセッサーには限られた数しかありません。AMD の EPYC 3 には 128 レーンがあり、Intel の Xeon にはチップあたり最大 48 レーンがあります。デスクトップグレードの CPU では、その数はそれぞれ 20 (Ryzen 9) と 16 (Core i9) です。通常、GPU には 16 レーンがあるため、全帯域幅で CPU に接続できる GPU の数が制限されます。結局のところ、ストレージやイーサネットなどの他の高帯域幅周辺機器とリンクを共有する必要があります。 RAM アクセスと同様に、パケット オーバーヘッドが削減されるため、大規模なバルク転送が推奨されます。
- **イーサネットは、**コンピュータを接続するために最も一般的に使用される方法です。 PCIe よりも大幅に遅いですが、非常に安価で設置が容易で、はるかに長い距離をカバーします。低グレードのサーバーの一般的な帯域幅は 1 GBit/s です。ハイエンド デバイス (クラウドの[C5 インスタンス](https://aws.amazon.com/ec2/instance-types/c5/)など) は、10 ～ 100 GBit/s の帯域幅を提供します。これまでのすべてのケースと同様に、データ送信には重大なオーバーヘッドがかかります。生のイーサネットを直接使用することはほとんどなく、物理的な相互接続 (UDP や TCP/IP など) 上で実行されるプロトコルを使用することに注意してください。これにより、さらにオーバーヘッドが追加されます。 PCIe と同様に、イーサネットは 2 つのデバイス (コンピュータとスイッチなど) を接続するように設計されています。
- **スイッチを使用すると、**任意のペアが (通常は全帯域幅で) ポイントツーポイント接続を同時に実行できる方法で、複数のデバイスを接続できます。たとえば、イーサネット スイッチは、高いクロスセクション帯域幅で 40 台のサーバーを接続する場合があります。スイッチは従来のコンピュータ ネットワークに特有のものではないことに注意してください。 PCIe レーンも[切り替える](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)ことができます。これは、たとえば、 [P2 インスタンス](https://aws.amazon.com/ec2/instance-types/p2/)の場合のように、多数の GPU をホスト プロセッサに接続する場合に発生します。
-  **NVLink は、**非常に高帯域幅の相互接続に関しては PCIe の代替手段です。リンクごとに最大 300 Gbit/s のデータ転送速度を実現します。サーバー GPU (Volta V100) には 6 つのリンクがありますが、コンシューマー グレードの GPU (RTX 2080 Ti) には 1 つのリンクしかなく、100 Gbit/s の速度で動作します。 GPU 間の高速データ転送を実現するには、 [NCCL](https://github.com/NVIDIA/nccl)を使用することをお勧めします。

## より多くのレイテンシ数値

:numref: `table_latency_numbers`および :numref: `table_latency_numbers_tesla`の概要は、数値の更新バージョンを[GitHub gist](https://gist.github.com/eshelman/343a1c46cb3fba142c1afdcdeec17646)として管理している[Eliot Eshelman](https://gist.github.com/eshelman)によるものです。

 :一般的なレイテンシの数値。

 |アクション |時間 |メモ | | :----------------------------------------------------- | -----: | :---------------------------------------------- | | L1 キャッシュ参照/ヒット | 1.5ns | 4サイクル | |浮動小数点加算/乗算/FMA | 1.5ns | 4サイクル | | L2キャッシュ参照/ヒット | 5ns | 12～17サイクル | |分岐の予測ミス | 6ns | 15～20サイクル | | L3 キャッシュ ヒット (非共有キャッシュ) | 16ns | 42サイクル | | L3 キャッシュ ヒット (別のコアで共有) | 25ns | 65サイクル | |ミューテックスのロック/ロック解除 | 25ns | | | L3 キャッシュ ヒット (別のコアで修正) | 29ns | 75サイクル | | L3 キャッシュ ヒット (リモート CPU ソケット上) | 40ns | 100 ～ 300 サイクル (40 ～ 116 ns) | |別の CPU への QPI ホップ (ホップごと) | 40ns | | | 64MB メモリ参照。 (ローカルCPU) | 46ns | Broadwell E5-2690v4 の TinyMemBench | | 64MB メモリ参照。 (リモートCPU) | 70ns | Broadwell E5-2690v4 の TinyMemBench | | 256MB メモリ参照。 (ローカルCPU) | 75ns | Broadwell E5-2690v4 の TinyMemBench | |インテル Optane ランダム書き込み | 94ns | UCSD 不揮発性システム研究所 | | 256MB メモリ参照。 (リモートCPU) | 120ns | Broadwell E5-2690v4 の TinyMemBench | |インテル Optane ランダム読み取り | 305ns | UCSD 不揮発性システム研究所 | | 100 Gbps HPC ファブリックで 4KB を送信 | 1μs | Intel Omni-Path 経由の MVAPICH2 | | Google Snappy で 1KB を圧縮 | 3μs | | | 10 Gbps イーサネット経由で 4KB を送信 | 10μs | | | NVMe SSD に 4KB をランダムに書き込む | 30μs | DC P3608 NVMe SSD (QOS 99% は 500μs) | | NVLink GPU との間で 1MB を転送 | 30μs | NVIDIA 40GB NVLink で最大 33GB/秒 | | PCI-E GPU との間で 1MB を転送 | 80μs | PCIe 3.0 x16 リンクで ~12GB/秒 | | NVMe SSD から 4KB をランダムに読み取る | 120μs | DC P3608 NVMe SSD (QOS 99%) | | NVMe SSD から 1MB を順次読み取ります。 208μs | ~4.8GB/秒 DC P3608 NVMe SSD | | SATA SSD に 4KB をランダムに書き込む | 500μs | DC S3510 SATA SSD (QOS 99.9%) | | SATA SSD から 4KB をランダムに読み取る | 500μs | DC S3510 SATA SSD (QOS 99.9%) | |同じデータセンター内の往復 | 500μs |一方向の ping は ~250μs | | SATA SSD から 1MB を順次読み取ります。 2ミリ秒 | ~550MB/s DC S3510 SATA SSD | |ディスクから 1MB を連続して読み取ります | 5ミリ秒 | ~200MB/秒のサーバー HDD | |ランダム ディスク アクセス (シーク+ローテーション) | 10ミリ秒 | | |パケットを送信 CA-&gt;オランダ-&gt;CA | 150ミリ秒 | | :label: `table_latency_numbers`

 :NVIDIA Tesla GPU のレイテンシー数値。

 |アクション |時間 |メモ | | :------------------------------ | -----: | :----------------------------------------------------- | | GPU 共有メモリ アクセス | 30ns | 30 ～ 90 サイクル (バンク競合によりレイテンシーが追加) | | GPU グローバル メモリ アクセス | 200ns | 200~800サイクル | | GPU で CUDA カーネルを起動 | 10μs |ホスト CPU が GPU にカーネルの起動を指示します。 | NVLink GPU との間で 1MB を転送 | 30μs | NVIDIA 40GB NVLink で最大 33GB/秒 | | PCI-E GPU との間で 1MB を転送 | 80μs | PCI-Express x16 リンクで ~12GB/秒 | :label: `table_latency_numbers_tesla`

## まとめ
- デバイスには操作のためのオーバーヘッドがあります。したがって、多数の小規模な転送ではなく、少数の大規模な転送を目指すことが重要です。これは、RAM、SSD、ネットワーク、GPU に当てはまります。
- ベクトル化はパフォーマンスの鍵となります。アクセラレータの特定の能力を必ず認識してください。たとえば、一部の Intel Xeon CPU は INT8 演算に特に適しており、NVIDIA Volta GPU は FP16 マトリックス間演算に優れており、NVIDIA Turing は FP16、INT8、および INT4 演算に優れています。
- データ型が小さいことによる数値オーバーフローは、トレーニング中 (および程度は低いですが推論中) に問題となる可能性があります。
- エイリアシングによりパフォーマンスが大幅に低下する可能性があります。たとえば、64 ビット CPU のメモリ アライメントは、64 ビット境界を基準にして行う必要があります。 GPU では、畳み込みサイズをテンソル コアなどに合わせて維持することをお勧めします。
- アルゴリズムをハードウェア (メモリ フットプリントや帯域幅など) に合わせます。パラメーターをキャッシュに適合させると、大幅な高速化 (桁違い) を達成できます。
- 実験結果を検証する前に、新しいアルゴリズムのパフォーマンスを紙に書き出してみることをお勧めします。一桁以上の不一致は懸念の理由です。
- プロファイラーを使用してパフォーマンスのボトルネックをデバッグします。
- トレーニングと推論のハードウェアには、価格とパフォーマンスの点で異なるスイート スポットがあります。

## 演習
1. C コードを作成して、外部メモリ インターフェイスに対してアライメントされたメモリとアライメントされていないメモリにアクセスする速度に違いがあるかどうかをテストします。ヒント: キャッシュ効果に注意してください。
1. メモリに順番にアクセスする場合と、一定のストライドでアクセスする場合の速度の違いをテストします。
1.  CPU 上のキャッシュ サイズを測定するにはどうすればよいでしょうか?
1. 帯域幅を最大化するには、複数のメモリ チャネルにデータをどのように配置しますか?小さな糸がたくさんある場合、どのように配置しますか?
1. エンタープライズクラスの HDD は 10,000 rpm で回転します。最悪の場合、HDD がデータを読み取るまでに必要な絶対最小時間はどれくらいですか (ヘッドがほぼ瞬時に移動すると想定できます)。商用サーバーでは (3.5 インチおよび 5.25 インチ ドライブと比較して) 2.5 インチ HDD が普及しているのはなぜですか?
1.  HDD メーカーがストレージ密度を 1 平方インチあたり 1 T ビットから 5 T ビット/平方インチに増加すると仮定します。 2.5 インチ HDD のリングにはどれくらいの情報を保存できますか? 内側のトラックと外側のトラックに違いはありますか?
1. データ型を 8 ビットから 16 ビットに変更すると、シリコンの量が約 4 倍に増加します。なぜ？ NVIDIA が Turing GPU に INT4 オペレーションを追加したのはなぜでしょうか?
1. メモリを順方向に読み取るのと、逆方向に読み取るのはどれくらい速いですか?この数値はコンピュータや CPU ベンダーによって異なりますか?なぜ？ C コードを作成して試してみましょう。
1. ディスクのキャッシュ サイズを測定できますか?一般的な HDD とは何ですか? SSDにはキャッシュが必要ですか?
1. イーサネット経由でメッセージを送信するときのパケット オーバーヘッドを測定します。 UDP 接続と TCP/IP 接続の違いを調べてください。
1. ダイレクト メモリ アクセスにより、CPU 以外のデバイスがメモリに (メモリから) 直接書き込み (および読み取り) できるようになります。なぜこれが良い考えなのでしょうか?
1.  Turing T4 GPU のパフォーマンス数値を見てください。 FP16 から INT8 および INT4 に移行すると、パフォーマンスが「だけ」2 倍になるのはなぜですか?
1. サンフランシスコとアムステルダム間の往復のパケットにかかる最短時間はどれくらいですか?ヒント: 距離は 10,000 km であると仮定できます。

[ディスカッション](https://discuss.d2l.ai/t/363)
